<a href="https://colab.research.google.com/github/elsebasantiago/Algoritmos_optimizacion_VIU/blob/main/Trabajo_Practico_Problema_del_doblaje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Práctico
# Sebastián Santiago
Link repositorio github
https://github.com/elsebasantiago/Algoritmos_optimizacion_VIU

In [1]:
import os
import pandas as pd
import numpy as np
from collections import deque
import random

ruta = os.path.join("" ,"Datos problema doblaje.csv")
datos = pd.read_csv(ruta, sep=',')
ds = datos.drop(['Unnamed: 11'], axis =1)

##Funciones Básicas

In [2]:
# generamos la fucnión de cálculo del costo
def costo_tomas(toma_origen, toma_destino):
  v_costo = toma_origen - toma_destino
  v_costo[v_costo > 0] = 0
  return sum(pow(element,2) for element in v_costo)


#Matriz de distancia entre tomas. Indica cuantos actores de diferencia tiene
# cada toma en la lista original (lista_tomas) versus una toma en particular
def matriz_distancia_toma(toma , lista_tomas):

  #Obtenemos la diferencia
  distancia_tomas = lista_tomas - toma

  #Los actores con valores negativos indican que están considerados en la toma
  # de comparación  por tanto, no los consideramos (igualamos a cero)
  # A su vez, se anulan los actores que participan en ambas tomas
  mask = distancia_tomas < 0
  distancia_tomas[mask] = 0

  #la suma de los actores restantes es la distancia entre las tomas
  return np.sum(distancia_tomas, axis = 1)

#Obtendremos el costo adicional que genera incorporar una nueva toma
def costo_adicional_toma (toma1, toma2):

  #creamos un vector diferencia entre tomas
  v_dif_tomas = toma2 - toma1

  #quitamos los valores negativos
  mask = v_dif_tomas <0
  v_dif_tomas[mask] = 0

  #sumamos los valores restantes para obtener el costo adicional
  #que incorporó la nueva toma
  return np.sum(v_dif_tomas)


#consideramos un actor mas a la lista de actores del día
def adiciona_actor(toma1, toma2):
  #obtenemos un vector que será la suma de los actores que participan
  #en ambas tomas
  v_sum_tomas = toma1 + toma2

  #normalizamos en 1 las posiciones de actores que coinciden
  mask = v_sum_tomas > 1
  v_sum_tomas[mask] = 1

  return v_sum_tomas

# Función que crea na solución aleatoria
def crea_solucion_aleatoria():
  return np.array(random.sample(range(1, 31), 30))



In [3]:
# creamos una función que devuelve los días, tomas y costos
def crear_dias_filmacion(lista_tomas, actores_en_toma):

  dias_tomas = np.zeros((len(lista_tomas),3)).astype(int)
  actores_toma_previa = np.zeros(actores_en_toma.shape[1])

  #Creamos una lista tipo fifo para ir recorriendo las tomas en la secuencia entregada
  lista_tomas_fifo = deque(lista_tomas.tolist())
  costo_total=0

  dia = 1
  j = 1
  i = 0

  while lista_tomas_fifo:

    # obtenemos el número de la primera toma
    toma = lista_tomas_fifo.popleft()
    #n_toma = lista_tomas[i]

    # obtenemos los actores que participan en la toma
    actores_toma_actual = actores_en_toma[toma-1]

    # Calculamos el costo de la toma, según la cantidad de actores
    costo_toma = np.sum(actores_toma_actual)
    costo_total += costo_toma

    # Matríz (día de toma, número de toma)z
    dias_tomas[i]= [dia, toma, costo_toma]

    #print(f'día = {dia} nro_toma = {toma} toma actual = {actores_toma_actual} costo = {costo_toma}')

    j=1

    while j < 6 and lista_tomas_fifo:

      #Tomamos la siguiente toma en la lista
      toma = lista_tomas_fifo.popleft()

      #Obtenemos los actores que deben participar en la siguiente toma en la lista
      actores_toma_siguiente = actores_en_toma[toma-1]

      #Calculamos el costo adiciona que conlleva considerar la nueva toma
      costo_toma = costo_adicional_toma(actores_toma_actual, actores_toma_siguiente)

      #Almacenamos el día, número de toma y costo adicional de la nueva toma
      dias_tomas[i+j]= [dia, toma, costo_toma]

      #print(f'dia = {dia} nro_toma = {toma} toma actual = {actores_toma_siguiente} costo = {costo_toma}')

      #Incluimos los nuevos actores a la lista de actores considerados para el día
      actores_toma_actual = adiciona_actor(actores_toma_actual,actores_toma_siguiente)
      costo_total += costo_toma
      j+=1

    i+=j
    dia +=1
  #print (f'costo total = {costo_total}')
  return dias_tomas


## Prueba de diferentes soluciones manuales

In [23]:
#Calculamos el costo de filmar las tomas en el orden presentado (de menor a mayor)

tomas_actores = np.array(datos[['1', '2','3','4', '5', '6', '7','8','9','10']]).copy()
tomas = np.array(datos['Toma']).copy()

solucion = crear_dias_filmacion(tomas, tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')



[[ 1  1  5]
 [ 1  2  0]
 [ 1  3  1]
 [ 1  4  1]
 [ 1  5  0]
 [ 1  6  0]
 [ 2  7  4]
 [ 2  8  1]
 [ 2  9  0]
 [ 2 10  1]
 [ 2 11  2]
 [ 2 12  0]
 [ 3 13  3]
 [ 3 14  2]
 [ 3 15  2]
 [ 3 16  1]
 [ 3 17  0]
 [ 3 18  0]
 [ 4 19  2]
 [ 4 20  2]
 [ 4 21  2]
 [ 4 22  1]
 [ 4 23  0]
 [ 4 24  0]
 [ 5 25  4]
 [ 5 26  3]
 [ 5 27  0]
 [ 5 28  0]
 [ 5 29  1]
 [ 5 30  0]]
solucion = 38


In [25]:
#Calculamos el costo de filmar las tomas en el orden inverso
#al presentado (de mayor a menor)

ds = datos.drop([ 'Total'], axis =1).copy()

ds =ds.sort_values(by='Toma', ascending=False)
tomas_actores = np.array(datos[['1', '2','3','4', '5', '6', '7','8','9','10']])
tomas = np.array(datos['Toma'])

solucion = crear_dias_filmacion(tomas, tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')


[[ 1  1  5]
 [ 1  2  0]
 [ 1  3  1]
 [ 1  4  1]
 [ 1  5  0]
 [ 1  6  0]
 [ 2  7  4]
 [ 2  8  1]
 [ 2  9  0]
 [ 2 10  1]
 [ 2 11  2]
 [ 2 12  0]
 [ 3 13  3]
 [ 3 14  2]
 [ 3 15  2]
 [ 3 16  1]
 [ 3 17  0]
 [ 3 18  0]
 [ 4 19  2]
 [ 4 20  2]
 [ 4 21  2]
 [ 4 22  1]
 [ 4 23  0]
 [ 4 24  0]
 [ 5 25  4]
 [ 5 26  3]
 [ 5 27  0]
 [ 5 28  0]
 [ 5 29  1]
 [ 5 30  0]]
solucion = 38


In [26]:
#Calculamos el costo de filmar las tomas ordenándolas primero con las
#escenas que demanan mayor cantidad de acotores y avanzando en orden decreciente

ds = datos.sort_values(by='Total', ascending=False)
ds = ds.drop(['Total'], axis =1)

tomas_actores = np.array(datos[['1', '2','3','4', '5', '6', '7','8','9','10']])
tomas = np.array(datos['Toma'])

solucion = crear_dias_filmacion(tomas, tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')


[[ 1  1  5]
 [ 1  2  0]
 [ 1  3  1]
 [ 1  4  1]
 [ 1  5  0]
 [ 1  6  0]
 [ 2  7  4]
 [ 2  8  1]
 [ 2  9  0]
 [ 2 10  1]
 [ 2 11  2]
 [ 2 12  0]
 [ 3 13  3]
 [ 3 14  2]
 [ 3 15  2]
 [ 3 16  1]
 [ 3 17  0]
 [ 3 18  0]
 [ 4 19  2]
 [ 4 20  2]
 [ 4 21  2]
 [ 4 22  1]
 [ 4 23  0]
 [ 4 24  0]
 [ 5 25  4]
 [ 5 26  3]
 [ 5 27  0]
 [ 5 28  0]
 [ 5 29  1]
 [ 5 30  0]]
solucion = 38


In [27]:
#Calculamos el costo de filmar las tomas ordenándolas primero con las
#escenas que demanan menor cantidad de acotores y avanzando en orden ascendente

ds = datos.sort_values(by='Total', ascending=True)
ds = ds.drop(['Total'], axis =1)

tomas_actores = np.array(datos[['1', '2','3','4', '5', '6', '7','8','9','10']])
tomas = np.array(datos['Toma'])

solucion = crear_dias_filmacion(tomas, tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')


[[ 1  1  5]
 [ 1  2  0]
 [ 1  3  1]
 [ 1  4  1]
 [ 1  5  0]
 [ 1  6  0]
 [ 2  7  4]
 [ 2  8  1]
 [ 2  9  0]
 [ 2 10  1]
 [ 2 11  2]
 [ 2 12  0]
 [ 3 13  3]
 [ 3 14  2]
 [ 3 15  2]
 [ 3 16  1]
 [ 3 17  0]
 [ 3 18  0]
 [ 4 19  2]
 [ 4 20  2]
 [ 4 21  2]
 [ 4 22  1]
 [ 4 23  0]
 [ 4 24  0]
 [ 5 25  4]
 [ 5 26  3]
 [ 5 27  0]
 [ 5 28  0]
 [ 5 29  1]
 [ 5 30  0]]
solucion = 38


## Búsqueda Aleatoria

In [34]:
# Solución con búsqueda aleatoria
def busqueda_aleatoria(actores_en_toma):

  # definimos a N como el número de iteraciones
  n = 1000
  mejor_costo = 10e100
  mejor_solucion = []

  while n > 0:

    lista_tomas = crea_solucion_aleatoria().copy()
    solucion = crear_dias_filmacion(lista_tomas, actores_en_toma)
    costo_actual = np.sum(solucion[:,2], axis = 0)

    if costo_actual < mejor_costo:
      mejor_solucion = solucion
      costo_actual = mejor_costo

    n-=1

  return mejor_solucion

tomas_actores = np.array(datos[['1', '2','3','4', '5', '6', '7','8','9','10']])
tomas = crea_solucion_aleatoria()

solucion = busqueda_aleatoria( tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')


[[ 1 22  4]
 [ 1 15  1]
 [ 1 16  1]
 [ 1  6  1]
 [ 1 23  0]
 [ 1 25  0]
 [ 2  2  3]
 [ 2 10  4]
 [ 2  4  2]
 [ 2 19  0]
 [ 2  8  0]
 [ 2 11  0]
 [ 3  5  3]
 [ 3 17  2]
 [ 3 30  0]
 [ 3  7  1]
 [ 3 29  1]
 [ 3  1  0]
 [ 4 27  2]
 [ 4 28  1]
 [ 4 18  2]
 [ 4 13  0]
 [ 4  3  2]
 [ 4 14  0]
 [ 5 20  4]
 [ 5  9  1]
 [ 5 12  1]
 [ 5 24  0]
 [ 5 26  1]
 [ 5 21  1]]
solucion = 38


##Búsqueda local

In [35]:
# Generamos solución vecina

def genera_vecina(lista_tomas,actores_en_toma):
  #Generador de soluciones vecinas mediante 2-opt

  mejor_solucion = []
  mejor_costo = 10e100

  for i in range(1,len(lista_tomas)-1):
    for j in range(i+1, len(lista_tomas)-1):

      # intercalamos posiciones
      lista_tomas[i], lista_tomas[j] =  + lista_tomas[j] , lista_tomas[i]

      #Se evalua la nueva solución
      solucion = crear_dias_filmacion(lista_tomas, actores_en_toma)
      costo_actual = np.sum(solucion[:,2], axis = 0)

      # Nos quedamos con la mejor solución
      if costo_actual < mejor_costo:
        mejor_solucion = solucion
        costo_actual = mejor_costo

  return mejor_solucion


In [38]:
# Solución con búsqueda local
def busqueda_local(actores_en_toma):

  solucion = busqueda_aleatoria(actores_en_toma)
  lista_tomas = solucion[:,1]
  mejor_costo = np.sum(solucion[:,2], axis = 0)
  mejor_solucion = solucion

  print(f'Solucion inicial = {lista_tomas}')
  print(f'costo inicial = {mejor_costo}')
  i = 10

  while i >0 :
    i-=1
    solucion = genera_vecina(lista_tomas,actores_en_toma)
    costo_tomas_vecina = np.sum(solucion[:,2], axis = 0)

    if costo_tomas_vecina < mejor_costo:
      mejor_solucion = solucion
      lista_tomas = solucion[:,1]
      mejor_costo = costo_tomas_vecina
    #else:
      #print(f'Solucion inicial = {mejor_solucion}')
  print(f'Iteraciones = {i} costo final = {mejor_costo}')
  return mejor_solucion

tomas_actores = np.array(datos[['1', '2','3','4', '5', '6', '7','8','9','10']])

solucion = busqueda_local(tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')

Solucion inicial = [28 22  2  9 16 19  5 15 11 24 30  8  3  4  1 13 10 27 14 21 29 26  7 18
 12  6 25 23 20 17]
costo inicial = 38
Iteraciones = 0 costo final = 38
[[ 1 28  2]
 [ 1 22  2]
 [ 1  2  1]
 [ 1  9  0]
 [ 1 16  1]
 [ 1 19  0]
 [ 2  5  3]
 [ 2 15  2]
 [ 2 11  2]
 [ 2 24  1]
 [ 2 30  0]
 [ 2  8  0]
 [ 3  3  3]
 [ 3  4  2]
 [ 3  1  2]
 [ 3 13  0]
 [ 3 10  2]
 [ 3 27  0]
 [ 4 14  3]
 [ 4 21  1]
 [ 4 29  1]
 [ 4 26  1]
 [ 4  7  2]
 [ 4 18  0]
 [ 5 12  5]
 [ 5  6  1]
 [ 5 25  1]
 [ 5 23  0]
 [ 5 20  0]
 [ 5 17  0]]
solucion = 38


## Solución Greedy

In [39]:
# Solución greedy
def crea_lista_greedy(lista_tomas, actores_en_toma):

  lista_tomas_disponibles = list(lista_tomas)
  resultado_tomas =[]

  toma = lista_tomas[0]
  lista_tomas_disponibles.remove(toma)
  actores_toma_actual = actores_en_toma[toma-1]
  m_distancia_a_tomas = matriz_distancia_toma(actores_toma_actual, actores_en_toma)
  #print(f'm_distancia_a_tomas = {m_distancia_a_tomas}')

  resultado_tomas.append(toma)
  m_distancia_a_tomas[toma-1] = 1000

  while len(lista_tomas_disponibles) > 0:
    toma_dist_menor = np.argmin(m_distancia_a_tomas) + 1

    if toma_dist_menor in lista_tomas_disponibles:
      resultado_tomas.append(toma_dist_menor)
      lista_tomas_disponibles.remove(toma_dist_menor)
      actores_toma_actual = adiciona_actor(actores_toma_actual,actores_en_toma[toma_dist_menor-1])
      m_distancia_a_tomas = matriz_distancia_toma(actores_toma_actual, actores_en_toma)
    else:
      m_distancia_a_tomas[toma_dist_menor-1] = 1000

  return np.array(resultado_tomas)


In [40]:
tomas_actores = np.array(datos[['1', '2','3','4', '5', '6', '7','8','9','10']])
tomas_original = np.array(datos['Toma'])

tomas = crea_lista_greedy(tomas_original, tomas_actores)

solucion = crear_dias_filmacion(tomas, tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')

[[ 1  1  5]
 [ 1  2  0]
 [ 1  6  0]
 [ 1  7  0]
 [ 1  9  0]
 [ 1 13  0]
 [ 2 17  2]
 [ 2 19  0]
 [ 2 20  2]
 [ 2 22  1]
 [ 2 23  0]
 [ 2 27  0]
 [ 3 28  2]
 [ 3 30  0]
 [ 3  3  3]
 [ 3 15  0]
 [ 3  4  1]
 [ 3  5  0]
 [ 4 11  5]
 [ 4  8  1]
 [ 4 12  1]
 [ 4 14  0]
 [ 4 18  0]
 [ 4 21  0]
 [ 5 24  2]
 [ 5 29  2]
 [ 5 10  2]
 [ 5 26  0]
 [ 5 16  2]
 [ 5 25  0]]
solucion = 31


In [45]:
tomas_actores = np.array(datos[['1', '2','3','4', '5', '6', '7','8','9','10']])
tomas_original = crea_solucion_aleatoria()

tomas = crea_lista_greedy(tomas_original, tomas_actores)

solucion = crear_dias_filmacion(tomas, tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')

[[ 1  6  4]
 [ 1  7  0]
 [ 1  9  0]
 [ 1 13  0]
 [ 1 27  0]
 [ 1 28  0]
 [ 2 30  2]
 [ 2  1  3]
 [ 2  2  0]
 [ 2 17  0]
 [ 2 19  0]
 [ 2 20  0]
 [ 3 22  4]
 [ 3 23  0]
 [ 3  3  2]
 [ 3 15  0]
 [ 3  4  1]
 [ 3  5  0]
 [ 4 11  5]
 [ 4  8  1]
 [ 4 12  1]
 [ 4 14  0]
 [ 4 18  0]
 [ 4 21  0]
 [ 5 24  2]
 [ 5 29  2]
 [ 5 10  2]
 [ 5 26  0]
 [ 5 16  2]
 [ 5 25  0]]
solucion = 31
